# Scrape data from the City of Peterborough website

In [340]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from typing import List, TypedDict


In [53]:
url = "https://www.peterborough.ca/en/city-hall/upcoming-and-past-agendas.aspx"

In [54]:
meetings_url = "https://pub-peterborough.escribemeetings.com/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2021"
post_url_2 = "https://pub-peterborough.escribemeetings.com/MeetingsCalendarView.aspx/PastMeetings?FillWidth=1&wmode=transparent&Year=2021&Expanded=Airport%20Strategic%20Initiatives%20Committee"

In [55]:
abc = requests.post(post_url_2)

In [60]:
abc.text[:300]

'<!DOCTYPE html>\r\n\r\n\r\n<html id="masterPageHeader" xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">\r\n<head><meta charset="utf-8" /><meta http-equiv="X-UA-Compatible" content="IE=edge;chrome=1" /><meta name="viewport" content="width=device-width, initial-scale=1.0" /><title>\r\n\teSCRIBE Publ'

So I need to find the links like `post_url_2` above in the `meetings_page`. And then do a post request and that should give me the list of urls for the minutes, agendas, videos, etc.

In [99]:
meetings_2021 = requests.get(meetings_url)

In [100]:
soup = BeautifulSoup(meetings_2021.text, 'html.parser')

In [101]:
meeting_type_anchors = soup.find_all("a", {"class": "PastMeetingTypesName"})

In [102]:
meeting_type_hrefs = [x.attrs["href"] for x in meeting_type_anchors]

In [110]:
meeting_type_hrefs

['https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2021&Expanded=Accessibility+Advisory+Committee+Meeting',
 'https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2021&Expanded=Ad+Hoc+Committee',
 'https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2021&Expanded=Airport+Strategic+Initiatives+Committee',
 'https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2021&Expanded=Arenas%2c+Parks+and+Recreation+Advisory+Committee+Meeting',
 'https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2021&Expanded=Arts%2c+Culture+and+Heritage+Advisory+Committee+Meeting',
 'https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2021&Expanded=Audit+Committee+Meetin

In [122]:
meeting_type_data_0 = requests.post(meeting_type_hrefs[0])

In [123]:
soup_0 = BeautifulSoup(meeting_type_data_0.text, 'html.parser')


In [124]:
soup_0.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html id="masterPageHeader" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head><meta charset="utf-8"/><meta content="IE=edge;chrome=1" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><title>
	eSCRIBE Published Meetings
</title><link href="https://cdnjs.cloudflare.com/ajax/libs/fullcalendar/3.0.1/fullcalendar.min.css" rel="stylesheet" type="text/css"/><link href="https://maxcdn.icons8.com/fonts/line-awesome/1.1/css/line-awesome.min.css" rel="stylesheet"/>
<script crossorigin="anonymous" integrity="sha256-CSXorXvZcTkaix6Yvo6HppcZGetbYMGWSFlBw8HfCJo=" src="https://code.jquery.com/jquery-3.4.1.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/moment.js/2.22.2/moment.min.js"></script>
<script crossorigin="anonymous" integrity="sha384-Q6E9RHvbIyZFJoft+2mJbHaEWldlvI9IOYy5n3zV9zzTtmI3UksdQRVvoxMfooAo" src="https://cdn.jsdelivr.net/npm/popper.js@1.16.0/dist/um

In [191]:
all_meetings_of_type_for_year = soup_0.find_all("div", {"class": "meeting-header"})

In [192]:
all_meetings_of_type_for_year

[<div class="meeting-header">
 <div class="meeting-title">
 <a aria-label="General Committee Monday, December 06, 2021" class="link" href="Meeting.aspx?Id=edf98604-03c7-4732-a5cf-e273976b77a3&amp;lang=English" target="_blank">General Committee</a>
 </div>
 <div class="pull-right wideVideo d-none">
 <a aria-label="Live Stream For General Committee 02 December 2021" class="link" href="VideoStream.aspx?MeetingId=edf98604-03c7-4732-a5cf-e273976b77a3" target="_blank">
 <i class="las la-video"></i>
                                                                 View Live Stream
                                                                 </a>
 </div>
 <div class="meeting-content">
 <div class="date-title">
 <div class="meeting-date"> Monday, December 06, 2021 @  6:30 PM</div>
 <div class="startLocation Location-Tooltip" data-container="body" data-html="true" data-original-title="Electronic Meeting" data-toggle="tooltip" style="word-break: break-word;">Electronic Meeting</div>
 <div clas

In [182]:
meeting_soup = all_meetings_of_type_for_year[0]

In [356]:
def parse_meeting_date(datetime_str: str) -> datetime:
    "datetime_str like Monday, 'December 06, 2021 @  6:30 PM'"
    string_format = "%A, %B %d, %Y @ %I:%M %p"
    return datetime.strptime(datetime_str, string_format)

class Meeting(TypedDict):
    datetime: datetime
    datetime_str: str
    meeting_type: str
    location: str
    links: List[Link]
    
class Link(TypedDict):
    aria_label: str
    link_text: str
    href: str
    url: str
    
    
def parse_link_soup(link_soup) -> Link:
    link: Link = {}
    link["aria_label"] = link_soup.attrs["aria-label"]
    link["href"] = link_soup.attrs["href"]
    link["link_text"] = link_soup.text.strip()
    link["url"] = "https://pub-peterborough.escribemeetings.com/" + link["href"]
    return link


def parse_meeting_soup(meeting_soup) -> Meeting:
    meeting: Meeting = {}
    meeting["meeting_type"] = meeting_soup.find("div", {"class": "meeting-title"}).text.strip()
    meeting["datetime_str"] = meeting_soup.find("div", {"class": "meeting-date"}).text.strip()
    meeting["location"] = meeting_soup.find("div", {"class": "startLocation"}).text.strip()
    meeting["datetime"] = parse_meeting_date(meeting["datetime_str"])    
    link_soup_list = meeting_soup.find_all("a", {"class": "link"})
    meeting["links"] = []
    for link_soup in link_soup_list:
        link = parse_link_soup(link_soup)
        meeting["links"].append(link)
    return meeting


In [355]:
parse_meeting_soup(meeting_soup)

{'meeting_type': 'Accessibility Advisory Committee Meeting',
 'datetime_str': 'Wednesday,  June 02, 2021 @  4:00 PM',
 'location': 'Electronic Meeting',
 'datetime': datetime.datetime(2021, 6, 2, 16, 0),
 'links': [{'aria_label': 'Accessibility Advisory Committee Meeting June 02, 2021',
   'href': 'Meeting.aspx?Id=329ee6cd-1ccf-4c62-9009-6f71b52b6cc2&lang=English',
   'link_text': 'Accessibility Advisory Committee Meeting'},
  {'aria_label': 'Minutes Accessibility Advisory Committee Meeting 6/2/2021 4:00:00 PM 02 June 2021',
   'href': 'FileStream.ashx?DocumentId=29894',
   'link_text': 'Minutes.pdf'},
  {'aria_label': 'Agenda (HTML) for Accessibility Advisory Committee Meeting 02 June 2021',
   'href': 'Meeting.aspx?Id=329ee6cd-1ccf-4c62-9009-6f71b52b6cc2&Agenda=Agenda&lang=English',
   'link_text': 'HTML'},
  {'aria_label': 'Agenda (PDF) for Accessibility Advisory Committee Meeting 02 June 2021',
   'href': 'FileStream.ashx?DocumentId=29176',
   'link_text': 'PDF'}]}

In [345]:
meeting: Meeting = {}
meeting["meeting_type"] = meeting_soup.find("div", {"class": "meeting-title"}).text.strip()
meeting["datetime_str"] = meeting_soup.find("div", {"class": "meeting-date"}).text.strip()
meeting["location"] = meeting_soup.find("div", {"class": "startLocation"}).text.strip()

In [346]:
meeting["datetime"] = parse_meeting_date(meeting["datetime_str"])

In [347]:
meeting

{'meeting_type': 'Accessibility Advisory Committee Meeting',
 'datetime_str': 'Wednesday,  June 02, 2021 @  4:00 PM',
 'location': 'Electronic Meeting',
 'datetime': datetime.datetime(2021, 6, 2, 16, 0)}

In [348]:
meeting_soup = all_meetings_of_type_for_year[6]

In [349]:
links_soup = meeting_soup.find_all("a", {"class": "link"})

In [351]:
links = []
for link_soup in links_soup:
    link: Link = {}
    link["aria_label"] = link_soup.attrs["aria-label"]
    link["href"] = link_soup.attrs["href"]
    link["link_text"] = link_soup.text.strip()
    links.append(link)

In [335]:
links

[{'resource_name': 'Accessibility Advisory Committee Meeting June 02, 2021',
  'href': 'Meeting.aspx?Id=329ee6cd-1ccf-4c62-9009-6f71b52b6cc2&lang=English',
  'link_text': 'Accessibility Advisory Committee Meeting'},
 {'resource_name': 'Minutes Accessibility Advisory Committee Meeting 6/2/2021 4:00:00 PM 02 June 2021',
  'href': 'FileStream.ashx?DocumentId=29894',
  'link_text': 'Minutes.pdf'},
 {'resource_name': 'Agenda (HTML) for Accessibility Advisory Committee Meeting 02 June 2021',
  'href': 'Meeting.aspx?Id=329ee6cd-1ccf-4c62-9009-6f71b52b6cc2&Agenda=Agenda&lang=English',
  'link_text': 'HTML'},
 {'resource_name': 'Agenda (PDF) for Accessibility Advisory Committee Meeting 02 June 2021',
  'href': 'FileStream.ashx?DocumentId=29176',
  'link_text': 'PDF'}]

In [326]:
("href", "aria-label")
x.attrs["href"]

'FileStream.ashx?DocumentId=29176'

In [327]:
x.attrs

{'href': 'FileStream.ashx?DocumentId=29176',
 'class': ['link'],
 'target': '_blank',
 'lang': 'en',
 'aria-label': 'Agenda (PDF) for Accessibility Advisory Committee Meeting 02 June 2021'}

In [328]:
x.text

'PDF'

In [302]:
i = 0

In [311]:
meeting_soup

<div class="meeting-header">
<div class="meeting-title">
<a aria-label="Accessibility Advisory Committee Meeting June 02, 2021" class="link" href="Meeting.aspx?Id=329ee6cd-1ccf-4c62-9009-6f71b52b6cc2&amp;lang=English" target="_blank">Accessibility Advisory Committee Meeting</a>
</div>
<div class="meeting-content">
<div class="date-title">
<div class="meeting-date"> Wednesday,  June 02, 2021 @  4:00 PM</div>
<div class="startLocation Location-Tooltip" data-container="body" data-html="true" data-original-title="Electronic Meeting" data-toggle="tooltip" style="word-break: break-word;">Electronic Meeting</div>
<div class="feature-icons">
<div class="tshare sharing"><a aria-label="Share Accessibility Advisory Committee Meeting" class="sharingIcon" href="Sharing.aspx?u=https%3A%2F%2Fpub-peterborough.escribemeetings.com%2FMeeting%3FId%3D329ee6cd-1ccf-4c62-9009-6f71b52b6cc2" title="Share">
<i class="fas fa-share-alt fa-lg"></i>
</a></div>
</div>
</div>
<div class="itemResources">
<div class="a

In [303]:
packageName = package_list[i].find("span", {"class": "packageName"}).text

In [304]:
packageName

'Agenda'

In [305]:
package_links = package_list[i].find_all("li", {"class": "resource-link"})

In [306]:
package_links

[<li class="resource-link">
 <a aria-label="Agenda (HTML) for Accessibility Advisory Committee Meeting 02 June 2021" class="link" href="Meeting.aspx?Id=329ee6cd-1ccf-4c62-9009-6f71b52b6cc2&amp;Agenda=Agenda&amp;lang=English" lang="en" target="_blank">HTML</a>
 </li>,
 <li class="resource-link">
 <a aria-label="Agenda (PDF) for Accessibility Advisory Committee Meeting 02 June 2021" class="link" href="FileStream.ashx?DocumentId=29176" lang="en" target="_blank">PDF</a>
 </li>]

In [307]:
link_soup = package_links[i].find("a")

In [308]:
link_soup.attrs["href"]


'Meeting.aspx?Id=329ee6cd-1ccf-4c62-9009-6f71b52b6cc2&Agenda=Agenda&lang=English'

In [309]:
link_soup.text

'HTML'

In [310]:
package_list

[<ul class="inline-package-collection">
 <li><i class="las la-file fa-lg"></i>
 <span class="packageName">Agenda</span>
 </li>
 <li class="resource-link">
 <a aria-label="Agenda (HTML) for Accessibility Advisory Committee Meeting 02 June 2021" class="link" href="Meeting.aspx?Id=329ee6cd-1ccf-4c62-9009-6f71b52b6cc2&amp;Agenda=Agenda&amp;lang=English" lang="en" target="_blank">HTML</a>
 </li>
 <li class="resource-link">
 <a aria-label="Agenda (PDF) for Accessibility Advisory Committee Meeting 02 June 2021" class="link" href="FileStream.ashx?DocumentId=29176" lang="en" target="_blank">PDF</a>
 </li>
 </ul>,
 <ul class="inline-package-collection">
 </ul>]

In [299]:
len(package_list)

2